In [10]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd 

In [11]:
def getHtml(querry, page):
    
    headers = {
        'authority': 'www.ebay.com',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8,hi;q=0.7,fr;q=0.6,gu;q=0.5,de;q=0.4',
        'dnt': '1',
        'referer': 'https://www.ebay.com/sch/i.html?_from=R40&_trksid=p4432023.m570.l1313&_nkw=mobile&_sacat=0',
        'sec-ch-ua': '"Not A(Brand";v="99", "Google Chrome";v="121", "Chromium";v="121"',
        'sec-ch-ua-full-version': '"121.0.6167.161"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-model': '""',
        'sec-ch-ua-platform': '"Windows"',
        'sec-ch-ua-platform-version': '"15.0.0"',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36',
    }

    params = {
        '_from': 'R40',
        '_nkw': querry,
        '_sacat': '0',
        '_pgn': page,
    }

    response = requests.get('https://www.ebay.com/sch/i.html', params=params, headers=headers)
    return response.text

In [12]:
def getpageData(querry,page):
    ItemList = []
    response = getHtml(querry, page)
    soup = BeautifulSoup(response, 'html.parser')
    AllProduct = soup.find_all('li', class_='s-item s-item__pl-on-bottom')
    for data in AllProduct:
        if AllProduct.index(data) == 0:
            continue
        try:
            productName  =  data.find('div', class_='s-item__title').text
        except:
            continue
        try :    
            productSubtitle = data.find('div', class_='s-item__subtitle').text
        except:
            productSubtitle = ''
        try:
            productRatingCount  = data.find('div', class_='x-star-rating').text
        except :
            productRatingCount = '0'
        try:
            ProductTotalRating = data.find('span', class_='s-item__reviews-count').text
        except:
            ProductTotalRating = '0'
        productLink = data.find('a', class_='s-item__link').get('href')
        ProductTotalRating = re.findall(r'^\d+', ProductTotalRating)[0]
        ProductPrice = data.find('span', class_='s-item__price').text
        productshiping=data.find('span',class_='s-item__itemLocation')
        if productshiping:
            productshiping=productshiping.text.strip()
        else:
            productshiping='location not available'
        productseller=data.find('span',class_='s-item__seller-info')
        if productseller:
            productseller=productseller.text.strip()
        else:
            productseller='seller not available'

        if ProductPrice:
            price = int(re.findall(r'\d+', ProductPrice)[0])
        if price> 100 and productRatingCount >= 100 :
            
            prouducts = {'productName': productName, 'productSubtitle': productSubtitle ,'ProductPrice': ProductPrice  , 'productRatingCount': productRatingCount, 'ProductTotalRating': ProductTotalRating , 'productshiping':productshiping,'productseller':productseller, 'productLink': productLink}
            ItemList.append(prouducts)
    return ItemList

In [18]:
totalPages =  1000
target_brands = [" Rockshox", "Shimano", "raceface"]
querry = "mountain biking"
TotalData = []
for brand in target_brands:
    page =1
    querry = querry + brand
    while page < totalPages :
        data = getpageData(querry, page)
        print(page)
        TotalData.extend(data)
        df = pd.DataFrame(TotalData)
        df.to_excel('ebay.xlsx', index=False)
        page+=1    

1
1
1


In [6]:
!pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 1.4 MB/s eta 0:00:0000:0100:01


In [9]:
df.to_csv('ebay.csv', index=False)

In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

def get_page_html(query, page):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'
    }

    params = {
        '_from': 'R40',
        '_nkw': query,
        '_sop': '12',  # Sort by Best Match
        '_ipg': '100',  # Items per page
        '_pgn': page
    }

    response = requests.get('https://www.ebay.com/sch/i.html', params=params, headers=headers)
    return response.text

def extract_data_from_page(html, target_brands):
    soup = BeautifulSoup(html, 'html.parser')
    all_products = soup.find_all('div', class_='s-item__info')

    data = []
    for product in all_products:
        brand_tags = product.find_all('span', class_='s-item__dynamic')
        brands = [tag.text.strip() for tag in brand_tags]

        # Check if the product has any of the desired brands
        if any(brand in brands for brand in target_brands):
            seller_name = product.find('span', class_='s-item__seller-info').text.strip()
            rating_count = int(re.search(r'\d+', product.find('span', class_='s-item__reviews-count').text.strip()).group())
            price = float(re.search(r'\d+\.\d+', product.find('span', class_='s-item__price').text.strip()).group())

            # Filter sellers with several hundred ratings and average item price over $100
            if rating_count >= 200 and price > 100:
                data.append({'SellerName': seller_name, 'RatingCount': rating_count, 'Price': price, 'Brands': brands})

    return data

def scrape_ebay(query, total_pages, target_brands):
    all_data = []
    for page in range(1, total_pages + 1):
        html = get_page_html(query, page)
        page_data = extract_data_from_page(html, target_brands)
        all_data.extend(page_data)
        print(f"Scraped page {page}")

    return all_data

if __name__ == "__main__":
    search_query = "mountain bike"
   
    total_pages_to_scrape = int(input("Enter the total number of pages to scrape: "))
    scraped_data = scrape_ebay(search_query, total_pages_to_scrape, target_brands)
    #Convert data to DataFrame
    dataframe = pd.DataFrame(scraped_data)
    # Save data to Excel
    dataframe.to_excel(f"{search_query}_ebay_data.xlsx", index=False)
    print("Scraping completed. Data saved to Excel.")


In [38]:
querry = "mountain biking"
page = 1
response = getHtml(querry, page)

In [39]:
with open('ebay.html', 'w', encoding='utf-8') as file:
    file.write(response)

In [40]:
soup = BeautifulSoup(response, 'html.parser')

In [41]:
last_page_link = soup.find_all('a', class_='pagination__item')

In [42]:
for i in last_page_link:
    print(i.text)

1
2
3
4
5
6
7
8
9
10
